Opcional --- Ensambles de modelos en el Lenguaje R
===

* 30:00 min | Ultima modificación: Junio 22, 2019

## Descripción del problema

Se desea construir un esquema de pronóstico que permita aprovechar la información capturada por muchos modelos diferentes construidos sobre los mismos datos.

Si se tienen múltiples pronósticos obtenidos por diferentes modelos para los mismos datos, es mejor simplemente seleccionar el mejor modelo y descartar la demás información, o se pueden aprovechar dichos pronósticos?

## Metodología
El problema se abordará desde las técnicas de ensamble de modelos que se abordaron en la sección teórica.

## Solución

In [1]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
%matplotlib inline
%load_ext rpy2.ipython

### Bagging (Bootstrap aggregation)

In [1]:
%%R
## 
## se carga la librería necesaria
##
library(ipred)
library(caret)

Loading required package: lattice
Loading required package: ggplot2


In [2]:
%%R
##
## Se cargan los datos
##
data <- read.csv("data/credit.csv")
data$default <- factor(data$default, labels=c("No", "Yes")) # se cambia la variable de decisión como factor

In [3]:
%%R
##
## Se entrena el modelo con 25 arboles
##
mybag <- bagging(default ~ ., data = data, nbagg = 25)
mybag  ## se muestra la salida del modelo


Bagging classification trees with 25 bootstrap replications 

Call: bagging.data.frame(formula = default ~ ., data = data, nbagg = 25)



In [4]:
%%R
##
## se predicen de acuerdo con el modelo construido
##
data_pred <- predict(mybag, data)
head(data_pred)

[1] No  Yes No  No  Yes No 
Levels: No Yes

In [7]:
%%R
## 
## Se crea la matriz de confusión
## note que se clasificaron incorrectamente solo 2 observaciones
##
table(data_pred, data$default)

         
data_pred  No Yes
      No  700   2
      Yes   0 298

In [6]:
%%R
##
## Para evaluar realmente la precisión del modelo
## se usa crossvalidation
##

set.seed(300)                                                        ## se establece la semilla aleatoria
ctrl <- trainControl(method = "cv", number = 10)                     ## se generan los parámetros
train(default ~ ., data = data, method = "treebag", trControl = ctrl)## se entrena el modelo

##
## note que tiene un desempeño muy similar al metodo C5.0
##

Bagged CART 

1000 samples
  20 predictor
   2 classes: 'No', 'Yes' 

No pre-processing
Resampling: Cross-Validated (10 fold) 
Summary of sample sizes: 900, 900, 900, 900, 900, 900, ... 
Resampling results:

  Accuracy  Kappa  
  0.746     0.36038


### AdaBoosting (Adaptive Boosting)

In [8]:
%%R
##
## se carga la librería necesaria
##
library(adabag)

Loading required package: rpart
Loading required package: foreach
Loading required package: doParallel
Loading required package: iterators
Loading required package: parallel

Attaching package: 'adabag'

The following object is masked from 'package:ipred':

    bagging



In [9]:
%%R
##
## se entrena el modelo
##
set.seed(300)                                    ## se establece la semilla aleatoria
m_adaboost <- boosting(default ~ ., data = data) ## se entrena el modelo con los parámetros por defecto
summary(m_adaboost)

           Length Class   Mode     
formula       3   formula call     
trees       100   -none-  list     
weights     100   -none-  numeric  
votes      2000   -none-  numeric  
prob       2000   -none-  numeric  
class      1000   -none-  character
importance   20   -none-  numeric  
terms         3   terms   call     
call          3   -none-  call     

In [10]:
%%R
##
## se realiza la predicción
##
p_adaboost <- predict(m_adaboost, data)

## se genera la matriz de confusión
p_adaboost$confusion

               Observed Class
Predicted Class  No Yes
            No  700   0
            Yes   0 300

Es posible utilizar el módulo de crossvalidation de la técnica para verificar los resultados.

In [11]:
%%R
##
## verificación usando CV
##
set.seed(300)
adaboost_cv <- boosting.cv(default ~ ., data = data) ## se entrena el modelo
adaboost_cv$confusion                                ## se realiza la matriz de confusión

i:  1 Tue Mar 26 10:54:00 2019 
i:  2 Tue Mar 26 10:54:36 2019 
i:  3 Tue Mar 26 10:55:12 2019 
i:  4 Tue Mar 26 10:55:48 2019 
i:  5 Tue Mar 26 10:56:24 2019 
i:  6 Tue Mar 26 10:56:59 2019 
i:  7 Tue Mar 26 10:57:35 2019 
i:  8 Tue Mar 26 10:58:13 2019 
i:  9 Tue Mar 26 10:58:50 2019 
i:  10 Tue Mar 26 10:59:25 2019 


               Observed Class
Predicted Class  No Yes
            No  594 153
            Yes 106 147

In [13]:
%%R
##
## se carga la librería necesaria
##
library(vcd)

In [14]:
%%R
##
## se realiza el análisis mediante Kappa de los resultados de la matriz
##
Kappa(adaboost_cv$confusion)

            value     ASE     z Pr(>|z|)
Unweighted 0.3544 0.03237 10.95 6.68e-28
Weighted   0.3544 0.03237 10.95 6.68e-28

### Random forest

In [15]:
%%R
##
## se carga la librería necesaria
##
library(randomForest)

randomForest 4.6-14
Type rfNews() to see new features/changes/bug fixes.

Attaching package: 'randomForest'

The following object is masked from 'package:ggplot2':

    margin



In [16]:
%%R
set.seed(300) ## se genera la semilla aleatoria

##
## por defecto crea un ensamble de 500 arboles
## que usan la raíz cuadrada de la cantidad
## de atributos presentes en el conjunto de entrenamiento
##
rf <- randomForest(default ~ ., data = data)
rf
## en la salida OOB se refiere a out-of-bag error rate


Call:
 randomForest(formula = default ~ ., data = data) 
               Type of random forest: classification
                     Number of trees: 500
No. of variables tried at each split: 4

        OOB estimate of  error rate: 23.2%
Confusion matrix:
     No Yes class.error
No  648  52  0.07428571
Yes 180 120  0.60000000

In [17]:
%%R
##
## A continuación se comparan los resultados de 
## random forest (rf) con C5.0 
##
library(caret)

##
## parametros de control
##
ctrl <- trainControl(method = "repeatedcv",
                     number = 10, 
                     repeats = 10)

##
## malla de parametros a considerar
##
grid_rf <- expand.grid(.mtry = c(2, 4, 8, 16))

set.seed(300)
m_rf <- train(default ~ ., 
              data = data, 
              method = "rf",
              metric = "Kappa", 
              trControl = ctrl,
              tuneGrid = grid_rf)
m_rf 

Random Forest 

1000 samples
  20 predictor
   2 classes: 'No', 'Yes' 

No pre-processing
Resampling: Cross-Validated (10 fold, repeated 10 times) 
Summary of sample sizes: 900, 900, 900, 900, 900, 900, ... 
Resampling results across tuning parameters:

  mtry  Accuracy  Kappa     
   2    0.7187    0.09466457
   4    0.7487    0.27624883
   8    0.7528    0.32307579
  16    0.7570    0.35357757

Kappa was used to select the optimal model using the largest value.
The final value used for the model was mtry = 16.

In [18]:
%%R
##
## comparación con un boosted tree
##
grid_c50 <- expand.grid(.model = "tree",
                        .trials = c(10, 20, 30, 40),
                        .winnow = "FALSE")
set.seed(300)
m_c50 <- train(default ~ ., 
               data = data, 
               method = "C5.0",
               metric = "Kappa", 
               trControl = ctrl,
               tuneGrid = grid_c50)
m_c50

Warning message in Ops.factor(x$winnow):
"'!' not meaningful for factors"

C5.0 

1000 samples
  20 predictor
   2 classes: 'No', 'Yes' 

No pre-processing
Resampling: Cross-Validated (10 fold, repeated 10 times) 
Summary of sample sizes: 900, 900, 900, 900, 900, 900, ... 
Resampling results across tuning parameters:

  trials  Accuracy  Kappa    
  10      0.7404    0.3389905
  20      0.7476    0.3557574
  30      0.7502    0.3623037
  40      0.7509    0.3645426

Tuning parameter 'model' was held constant at a value of tree
Tuning
 parameter 'winnow' was held constant at a value of FALSE
Kappa was used to select the optimal model using the largest value.
The final values used for the model were trials = 40, model = tree and winnow
 = FALSE.